In [ ]:
import warnings
import torch
import pandas as pd
import numpy as np
import scanpy as sc
import os
import random
from torch.backends import cudnn
import episcanpy.api as epi
from PRESENT import PRESENT_function

warnings.filterwarnings("ignore")
sc.settings.set_figure_params(dpi_save=300, dpi=120, fontsize=10, transparent=True, figsize =(3,2.5), format='pdf')

In [ ]:
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.enabled=False
    cudnn.benchmark = False
    torch.use_deterministic_algorithms(True,warn_only=True)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' 

In [ ]:
set_seed(42)

In [ ]:
adata_adt = sc.read_h5ad("./data/10x Visium human tonsil/protein.h5ad")
print(adata_adt)

adata_rna = sc.read_h5ad("./data/10x Visium human tonsil/rna.h5ad")
print(adata_rna)

loc = pd.read_csv('./data/10x Visium human tonsil/locs.csv')
spatial_coords = loc.iloc[:, [2, 3]].values  
adata_rna.obsm['spatial'] = spatial_coords
adata_adt.obsm['spatial'] = spatial_coords

In [ ]:
adata = PRESENT_function(
    spatial_key = "spatial", ## obsm key under which to load the spatial matrix of spots
    adata_rna = adata_rna, ## The RNA raw fragment count matrix of spots in anndata.AnnData format
    gene_min_cells = 1, ## Minimum number of cells expressed required for a gene to pass filtering
    num_hvg = 3000, ## Number of highly variable genes to select for RNA data
    adata_adt = adata_adt, ## The ADT raw fragment count matrix of spots in anndata.AnnData format
    protein_min_cells = 1, ## Minimum number of cells expressed required for a protein to pass filtering
    nclusters = 10,
    device = "cuda" ## Device used for training: cuda or cpu
)
print(adata)

In [ ]:
sc.pl.embedding(adata, basis="spatial", color= "LeidenClusters", wspace=0.5)